In [7]:
from pole_gen.pole_gen import generate_utility_pole_mesh, generate_road_meshes
from utils.plot import plot_open3d
import secrets
import numpy as np

seed = secrets.randbits(32)
np.random.seed(seed)
print(f"Seed: {seed}")

state = generate_utility_pole_mesh()

# Add some road meshes for debugging
road_meshes = generate_road_meshes(state)
road_meshes.rotate(
    state.geometry.get_rotation_matrix_from_xyz((0, 0, state.z_rotation)),
    center=(0, 0, 0),
)

plot_open3d([state.geometry, road_meshes])

Seed: 1802027995


In [34]:
from pole_gen.raycasting import scan_geometry
from utils.plot import plot_open3d

npoints = 500  # np.random.randint(800, 103000)

pc = scan_geometry(state, npoints=npoints, sensor_pos=(3, 2, 1))
print(f"Number of points: {len(pc.points)}")

plot_open3d([pc])

Number of points: 500
